# Building tools for agents

In [109]:
import pandas as pd
import random as random
from datetime import datetime

#  Flight Schedule 
flight_schedule_df = pd.DataFrame([
    {
        "flight_id": "UA123",
        "origin": "ORD",
        "destination": "SFO",
        "sched_dep": "2024-08-10 08:00",
        "sched_arr": "2024-08-10 11:00",
        "aircraft_type": "B737",
        "delay_minutes": 210,  # 3.5 hr delay
        "status": "delayed",
        "gate": "C5",
        "remarks": "ground stop"
    },
    {
        "flight_id": "UA456",
        "origin": "SFO",
        "destination": "DEN",
        "sched_dep": "2024-08-10 12:00",
        "sched_arr": "2024-08-10 14:00",
        "aircraft_type": "B737",
        "delay_minutes": 0,
        "status": "ontime",
        "gate": "B12",
        "remarks": ""
    }
])

#  Crew Roster 
crew_roster_df = pd.DataFrame([
    {
        "crew_id": "C001",
        "name": "Jane Doe",
        "role": "captain",
        "base": "ORD",
        "qualified_aircraft": "B737",
        "assigned_flight_id": "UA123",
        "duty_start": "2024-08-10 07:00",
        "duty_end": "2024-08-10 15:00",
        "rest_until": "2024-08-10 23:00",
        "status": "active"
    },
    {
        "crew_id": "C002",
        "name": "John Roe",
        "role": "FO",
        "base": "ORD",
        "qualified_aircraft": "B737",
        "assigned_flight_id": "UA123",
        "duty_start": "2024-08-10 07:00",
        "duty_end": "2024-08-10 15:00",
        "rest_until": "2024-08-10 23:00",
        "status": "active"
    },
    {
        "crew_id": "C010",
        "name": "Sam Lee",
        "role": "FO",
        "base": "SFO",
        "qualified_aircraft": "B737",
        "assigned_flight_id": None,
        "duty_start": None,
        "duty_end": None,
        "rest_until": "2024-08-10 06:00",
        "status": "active"
    },
    {
        "crew_id": "C011",
        "name": "Alex Kim",
        "role": "FO",
        "base": "DEN",
        "qualified_aircraft": "B737",
        "assigned_flight_id": None,
        "duty_start": None,
        "duty_end": None,
        "rest_until": "2024-08-10 04:00",
        "status": "active"
    }
])

#  Hotel Inventory 
hotel_inventory_df = pd.DataFrame([
    {
        "hotel_id": "H001",
        "name": "ORD Airport Hotel",
        "location": "ORD",
        "capacity": 200,
        "available_rooms": 0
    },
    {
        "hotel_id": "H002",
        "name": "ORD Downtown Inn",
        "location": "ORD",
        "capacity": 150,
        "available_rooms": 10
    },
    {
        "hotel_id": "H003",
        "name": "SFO Airport Hotel",
        "location": "SFO",
        "capacity": 100,
        "available_rooms": 5
    },
    {
        "hotel_id": "H004",
        "name": "DEN Airport Hotel",
        "location": "DEN",
        "capacity": 120,
        "available_rooms": 8
    }
])

#  Repositioning Flights 
repositioning_flights_df = pd.DataFrame([
    {
        "flight_id": "UA9001",
        "origin": "DEN",
        "destination": "SFO",
        "sched_dep": "2024-08-10 10:00",
        "sched_arr": "2024-08-10 12:00",
        "seats_available": True
    },
    {
        "flight_id": "UA9002",
        "origin": "DEN",
        "destination": "ORD",
        "sched_dep": "2024-08-10 11:00",
        "sched_arr": "2024-08-10 13:00",
        "seats_available": False
    },
    {
        "flight_id": "UA9003",
        "origin": "SFO",
        "destination": "ORD",
        "sched_dep": "2024-08-10 09:00",
        "sched_arr": "2024-08-10 15:00",
        "seats_available": True
    }
])

In [110]:
def query_flight_status(flight_id, flight_schedule_df):
    """
    Return status information for the given flight ID.
    
    Args:
        flight_id (str): Flight identifier.
        flight_schedule_df (DataFrame): Flight schedule data source.
    
    Returns:
        dict: { flight_id, status, delay_minutes, gate, remarks, aircraft_type }
    """
    # Step 1: Filter DataFrame for matching flight_id
    flight = flight_schedule_df[flight_schedule_df['flight_id'] == flight_id]
    
    # Step 2: Check if flight exists
    if flight.empty:
        return {"error": f"Flight {flight_id} not found"}
    
    # Step 3: Extract the first (and only) matching row
    flight_data = flight.iloc[0]
    
    # Step 4: Return relevant flight information
    return {
    "flight_id": flight_data['flight_id'],
    "status": flight_data['status'], 
    "delay_minutes": flight_data['delay_minutes'],
    "aircraft_type": flight_data['aircraft_type'],  # For crew qualifications
    "origin": flight_data['origin'],                # For crew location
    "destination": flight_data['destination'],      # For crew location  
    "remarks": flight_data['remarks']               # For delay context
    }

In [111]:
def query_crew_roster(flight_id=None, crew_id=None, crew_roster_df=None):
    """
    Fetch assigned crew for a flight or details for a specific crew member.
    
    Args:
        flight_id (str, optional): Flight identifier.
        crew_id (str, optional): Crew identifier.
        crew_roster_df (DataFrame): Crew data source.
    
    Returns:
        list of dicts or dict: Crew details.
    """
    if flight_id:
        crew = crew_roster_df[crew_roster_df['assigned_flight_id'] == flight_id]
        return crew.to_dict('records')
    
    elif crew_id:
        crew = crew_roster_df[crew_roster_df['crew_id'] == crew_id]
        if crew.empty:
            return {"error": f"Crew {crew_id} not found"}
        return crew.iloc[0].to_dict()
    
    else:
        return {"error": "Must specify either flight_id or crew_id"}

In [112]:
def query_spare_pool(location, aircraft_type, role, crew_roster_df):
    """
    Find available spare crew at location with given qualification.
    
    Args:
        location (str)
        aircraft_type (str)
        role (str)
        crew_roster_df (DataFrame)
    
    Returns:
        list of dicts: Spare crew candidates.
    """
    # Filter for spare crew: no assigned flight, active status, at location, qualified
    spare_crew = crew_roster_df[
        (crew_roster_df['assigned_flight_id'].isna()) &
        (crew_roster_df['status'] == 'active') &
        (crew_roster_df['base'] == location) &
        (crew_roster_df['qualified_aircraft'].str.contains(aircraft_type, na=False)) &
        (crew_roster_df['role'] == role)
    ]
    
    # Check if rest period has ended (assuming current time is 2024-08-10 08:00 for demo)
    current_time = datetime.strptime("2024-08-10 08:00", "%Y-%m-%d %H:%M")
    available_crew = []
    
    for _, crew in spare_crew.iterrows():
        rest_until = datetime.strptime(crew['rest_until'], "%Y-%m-%d %H:%M")
        if current_time >= rest_until:
            available_crew.append(crew.to_dict())
    
    return available_crew

In [113]:
def duty_hour_checker(crew_id, planned_start, planned_end, duty_rules, crew_roster_df):
    """
    Check legality of crew duty period.
    
    Args:
        crew_id (str)
        planned_start (str): datetime string
        planned_end (str): datetime string  
        duty_rules (dict): {'max_duty_hours': 8, 'min_rest_hours': 10}
        crew_roster_df (DataFrame)
    
    Returns:
        dict: { legal: bool, reason: str if not legal, remaining_duty_minutes: int }
    """
    crew = crew_roster_df[crew_roster_df['crew_id'] == crew_id]
    if crew.empty:
        return {"legal": False, "reason": f"Crew {crew_id} not found"}
    
    crew_data = crew.iloc[0]
    
    # Parse times
    start_time = datetime.strptime(planned_start, "%Y-%m-%d %H:%M")
    end_time = datetime.strptime(planned_end, "%Y-%m-%d %H:%M")
    
    # Calculate duty duration
    duty_duration = (end_time - start_time).total_seconds() / 3600  # hours
    max_duty = duty_rules.get('max_duty_hours', 8)
    
    if duty_duration > max_duty:
        return {
            "legal": False, 
            "reason": f"Duty period {duty_duration:.1f}h exceeds maximum {max_duty}h",
            "remaining_duty_minutes": 0
        }
    
    # Check rest requirements if crew has previous duty
    if pd.notna(crew_data['duty_end']):
        prev_duty_end = datetime.strptime(crew_data['duty_end'], "%Y-%m-%d %H:%M")
        rest_duration = (start_time - prev_duty_end).total_seconds() / 3600
        min_rest = duty_rules.get('min_rest_hours', 10)
        
        if rest_duration < min_rest:
            return {
                "legal": False,
                "reason": f"Rest period {rest_duration:.1f}h below minimum {min_rest}h",
                "remaining_duty_minutes": 0
            }
    
    remaining_minutes = int((max_duty - duty_duration) * 60)
    return {
        "legal": True, 
        "reason": "Within duty limits",
        "remaining_duty_minutes": remaining_minutes
    }

In [114]:
def reposition_flight_finder(from_location, to_location, repositioning_flights_df):
    """
    Find repositioning flight options for crew.
    
    Args:
        from_location (str)
        to_location (str)
        repositioning_flights_df (DataFrame)
    
    Returns:
        list of dicts: Flight options.
    """
    reposition_options = repositioning_flights_df[
        (repositioning_flights_df['origin'] == from_location) &
        (repositioning_flights_df['destination'] == to_location) &
        (repositioning_flights_df['seats_available'] == True)
    ]
    
    return reposition_options.to_dict('records')

In [115]:
def book_hotel(location, crew_id, hotel_inventory_df):
    """
    Attempt to book hotel for crew.
    
    Args:
        location (str)
        crew_id (str)
        hotel_inventory_df (DataFrame)
    
    Returns:
        dict: { success: bool, hotel_id or failure_reason }
    """
    available_hotels = hotel_inventory_df[
        (hotel_inventory_df['location'] == location) &
        (hotel_inventory_df['available_rooms'] > 0)
    ].sort_values('available_rooms', ascending=False)
    
    if available_hotels.empty:
        return {
            "success": False,
            "failure_reason": f"No available rooms at {location}"
        }
    
    # Book at hotel with most availability
    hotel = available_hotels.iloc[0]
    
    # Simulate booking (would update database in real system)
    return {
        "success": True,
        "hotel_id": hotel['hotel_id'],
        "hotel_name": hotel['name'],
        "confirmation": f"CONF-{crew_id}-{random.randint(1000, 9999)}"
    }

In [ ]:
# Install dependencies (run once)
# !pip install sentence-transformers faiss-cpu

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Define your policies
policy_docs = [
    {
        "id": "no_legal_crew",
        "title": "No Legal Crew Available",
        "text": "If no legal crew can be found due to rest or duty violations, escalate to ops center. Contact duty manager for extension approval."
    },
    {
        "id": "hotel_overflow",
        "title": "Hotel Overflow",
        "text": "In case of full hotel capacity, crew must be booked in backup lodging within 50 miles. Notify crew scheduling immediately."
    },
    {
        "id": "weather_delay",
        "title": "Weather Delay Protocol",
        "text": "For weather delays, crew duty may be extended by up to 2 hours with manager approval."
    },
    {
        "id": "maintenance_delay",
        "title": "Maintenance Delay",
        "text": "If maintenance exceeds 4 hours, crew should be reassigned. Begin spare crew search and passenger comms."
    }
]

# Load embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Embed all policy texts
policy_texts = [doc["text"] for doc in policy_docs]
policy_embeddings = model.encode(policy_texts, convert_to_numpy=True)

# RAG-style policy retriever
def rag_policy_retriever(query: str) -> dict:
    query_embedding = model.encode([query], convert_to_numpy=True)
    similarities = cosine_similarity(query_embedding, policy_embeddings)[0]
    best_idx = int(np.argmax(similarities))
    best_policy = policy_docs[best_idx]
    
    return {
        "policy_id": best_policy["id"],
        "title": best_policy["title"],
        "policy_text": best_policy["text"],
        "score": float(similarities[best_idx])
    }

ValueError: Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

In [117]:
def check_aircraft_assignment_change(flight_id, flight_schedule_df):
    """
    Detect if aircraft type has changed for flight.
    
    Args:
        flight_id (str)
        flight_schedule_df (DataFrame)
    
    Returns:
        dict: { aircraft_type, changed: bool }
    """
    # In real system, would compare against historical data
    # For demo, randomly simulate aircraft changes
    flight = flight_schedule_df[flight_schedule_df['flight_id'] == flight_id]
    
    if flight.empty:
        return {"error": f"Flight {flight_id} not found"}
    
    current_aircraft = flight.iloc[0]['aircraft_type']
    
    # Mock 5% chance of aircraft change
    changed = random.random() < 0.05
    
    return {
        "aircraft_type": current_aircraft,
        "changed": changed,
        "previous_aircraft": "A320" if changed else current_aircraft
    }


In [118]:
def check_weather_conditions(airport_code, weather_data=None):
    """
    Get weather summary and delay risk.
    
    Args:
        airport_code (str)
        weather_data (dict or DataFrame, optional)
    
    Returns:
        dict: { summary, risk_level }
    """
    # Mock weather conditions
    weather_scenarios = {
        "ORD": {"summary": "Clear skies", "risk_level": "low"},
        "SFO": {"summary": "Fog forming", "risk_level": "high"}, 
        "DEN": {"summary": "Light snow", "risk_level": "medium"},
        "LAX": {"summary": "Sunny", "risk_level": "low"}
    }
    
    return weather_scenarios.get(airport_code, {
        "summary": "Conditions unknown", 
        "risk_level": "medium"
    })

In [119]:
def check_crew_future_assignment(crew_id, crew_roster_df):
    """
    Check future assignment conflicts for spare crew.
    
    Args:
        crew_id (str)
        crew_roster_df (DataFrame)
    
    Returns:
        dict: { next_assignment, conflict: bool }
    """
    # Mock future assignments check
    # In real system, would query crew scheduling database
    
    crew = crew_roster_df[crew_roster_df['crew_id'] == crew_id]
    if crew.empty:
        return {"error": f"Crew {crew_id} not found"}
    
    # Simulate 20% chance of future conflict
    has_conflict = random.random() < 0.2
    
    if has_conflict:
        return {
            "next_assignment": f"UA{random.randint(100, 999)}",
            "conflict": True,
            "conflict_time": "2024-08-10 18:00"
        }
    else:
        return {
            "next_assignment": None,
            "conflict": False
        }


In [120]:
def send_notification(message_text, priority):
    """
    Send structured notification to ops team.
    
    Args:
        message_text (str)
        priority (str): "low", "medium", "high", "critical"
    
    Returns:
        dict: { sent: bool, timestamp }
    """
    # Mock notification system
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # Simulate 95% success rate
    success = random.random() < 0.95
    
    if success:
        return {
            "sent": True,
            "timestamp": timestamp,
            "message_id": f"MSG-{random.randint(10000, 99999)}",
            "priority": priority
        }
    else:
        return {
            "sent": False,
            "timestamp": timestamp,
            "error": "Notification system temporarily unavailable"
        }

# Duty rules configuration
DUTY_RULES = {
    "max_duty_hours": 8,
    "min_rest_hours": 10,
    "extension_max_hours": 2,  # Max extension with approval
    "emergency_extension_hours": 4  # Absolute maximum
}

# Testing

In [121]:
# Synthetic flight schedule for testing
test_flights = pd.DataFrame([
    {
        "flight_id": "TEST100",
        "origin": "ORD",
        "destination": "LAX",
        "sched_dep": "2025-07-01 10:00",
        "sched_arr": "2025-07-01 13:00",
        "aircraft_type": "A320",
        "delay_minutes": 90,
        "status": "delayed",
        "gate": "B7",
        "remarks": "mechanical"
    },
    {
        "flight_id": "TEST200",
        "origin": "LAX",
        "destination": "JFK",
        "sched_dep": "2025-07-01 14:00",
        "sched_arr": "2025-07-01 22:00",
        "aircraft_type": "B777",
        "delay_minutes": 0,
        "status": "ontime",
        "gate": "A1",
        "remarks": ""
    }
])

# Basic assertions
assert query_flight_status("TEST100", test_flights) == {
    "flight_id": "TEST100",
    "status": "delayed",
    "delay_minutes": 90,
    "aircraft_type": "A320",
    "origin": "ORD",
    "destination": "LAX",
    "remarks": "mechanical"
}

assert query_flight_status("FAKE999", test_flights) == {
    "error": "Flight FAKE999 not found"
}

print("All tests passed.")


All tests passed.


In [127]:
# Run corrected unit tests
ua123_crew = crew_roster_df[crew_roster_df['assigned_flight_id'] == 'UA123'].to_dict('records')
assert query_crew_roster(flight_id="UA123", crew_roster_df=crew_roster_df) == ua123_crew

c001_info = crew_roster_df[crew_roster_df['crew_id'] == 'C001'].iloc[0].to_dict()
assert query_crew_roster(crew_id="C001", crew_roster_df=crew_roster_df) == c001_info

assert query_crew_roster(crew_id="C999", crew_roster_df=crew_roster_df) == {
    "error": "Crew C999 not found"
}

assert query_crew_roster(crew_roster_df=crew_roster_df) == {
    "error": "Must specify either flight_id or crew_id"
}

print("All fixed crew roster tests passed.")

All fixed crew roster tests passed.


In [ ]:
# Test crew roster
crew_roster_df = pd.DataFrame([
    {"crew_id": "C001", "duty_end": "2025-07-01 14:00"},
    {"crew_id": "C002", "duty_end": None}
])

# Duty rules
duty_rules = {"max_duty_hours": 8, "min_rest_hours": 10}

# Test 1: Valid duty and rest
assert duty_hour_checker("C002", "2025-07-01 10:00", "2025-07-01 17:00", duty_rules, crew_roster_df) == {
    "legal": True,
    "reason": "Within duty limits",
    "remaining_duty_minutes": 60
}

# Test 2: Duty too long
assert duty_hour_checker("C002", "2025-07-01 10:00", "2025-07-01 20:00", duty_rules, crew_roster_df) == {
    "legal": False,
    "reason": "Duty period 10.0h exceeds maximum 8h",
    "remaining_duty_minutes": 0
}

# Test 3: Too short rest
assert duty_hour_checker("C001", "2025-07-01 18:00", "2025-07-01 22:00", duty_rules, crew_roster_df) == {
    "legal": False,
    "reason": "Rest period 4.0h below minimum 10h",
    "remaining_duty_minutes": 0
}

# Test 4: Crew not found
assert duty_hour_checker("C999", "2025-07-01 10:00", "2025-07-01 15:00", duty_rules, crew_roster_df) == {
    "legal": False,
    "reason": "Crew C999 not found"
}

print("All duty hour checker tests passed.")

All duty hour checker tests passed.


In [ ]:
# Assertions
assert reposition_flight_finder("DEN", "SFO", repositioning_flights_df) == [
    repositioning_flights_df.iloc[0].to_dict()
]
assert reposition_flight_finder("DEN", "ORD", repositioning_flights_df) == []

assert reposition_flight_finder("SFO", "ORD", repositioning_flights_df) == [
    repositioning_flights_df.iloc[2].to_dict()
]
print("All reposition flight finder tests passed.")

All reposition flight finder tests passed.


In [ ]:
# Test ORD booking
result_ord = book_hotel("ORD", "C001", hotel_inventory_df)
assert result_ord["success"] == True
assert result_ord["hotel_id"] == "H002"
assert "CONF-C001-" in result_ord["confirmation"]

# Test SFO booking
result_sfo = book_hotel("SFO", "C002", hotel_inventory_df)
assert result_sfo["success"] == True
assert result_sfo["hotel_id"] == "H003"
assert "CONF-C002-" in result_sfo["confirmation"]

# Test DEN booking
result_den = book_hotel("DEN", "C003", hotel_inventory_df)
assert result_den["success"] == True
assert result_den["hotel_id"] == "H004"
assert "CONF-C003-" in result_den["confirmation"]

print("All hotel booking tests passed.")

All hotel booking tests passed.
